# Convert variables stored in JSON files to LaTeX variables

In [1]:
from IPython.display import display
from pathlib import Path
import json
import xarray as xr

from boost_and_layouts import (
    create_latex_variable,
    create_loglist,
    list_to_latex,
    format_variable_name,
    format_power_of_ten,
)

In [2]:
def parameters_to_latex(parameters, survey):
    """
    Generate a list of latex variables from a set of parameters
    """
    parameters_to_latex = []

    for layout in parameters:
        for depth_type in parameters[layout]:
            for parameter in parameters[layout][depth_type]:

                if parameter == "depth_type":
                    continue
                elif parameter == "damping":
                    values = create_loglist(parameters[layout][depth_type][parameter])
                else:
                    values = list_to_latex(parameters[layout][depth_type][parameter])
                variable_name = format_variable_name(
                    "_".join([survey, layout, depth_type, parameter])
                )
                parameters_to_latex.append(
                    r"\newcommand{{\{variable_name}}}{{{values}}}".format(
                        variable_name=variable_name, values=values
                    )
                )
    return parameters_to_latex


def format_damping(variable_name, value):
    """
    Convert damping to a LaTeX variable
    """
    variable_name = format_variable_name(variable_name)
    value = format_power_of_ten(value)
    return r"\newcommand{{\{variable_name}}}{{{value}}}".format(
        variable_name=variable_name, value=value
    )


def best_parameters_to_latex(parameters, survey):
    """
    Convert best parameters to LaTeX variables

    Parameters
    ----------
    parameters : dict
        Dictionary containing the parameters of the best prediction.
    survey : str
        Name of the gridded survey. Eg. ``"ground"``, ``"airborne"``.
    """
    latex_variables = []
    layout = parameters["layout"]
    depth_type = parameters["depth_type"]
    for key, value in parameters.items():
        if key in ["metadata", "depth_type", "layout"]:
            continue

        variable_name = "_".join(["best", survey, layout, depth_type, key])
        if key == "damping":
            variable = format_damping(variable_name, value)
        elif key == "rms":
            variable = create_latex_variable(variable_name, value, fmt=".2f")
        else:
            variable = create_latex_variable(variable_name, value)
        latex_variables.append(variable)
    return latex_variables

/home/santi/.anaconda3/envs/eql-gradient-boosted/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Define results and manuscript directories

In [3]:
results_dir = Path("..") / "results"
manuscript_dir = Path("..") / "manuscript" / "results"
ground_results_dir = results_dir / "ground_survey"
airborne_results_dir = results_dir / "airborne_survey"

## Source layouts schematics

In [4]:
json_file = results_dir / "source-layouts-schematics.json"
tex_file = manuscript_dir / "source-layouts-schematics.tex"

with open(json_file, "r") as f:
    variables = json.loads(f.read())

In [5]:
latex_variables = [
    create_latex_variable(key, value) for key, value in variables.items()
]
with open(tex_file, "w") as f:
    f.write("\n".join(latex_variables))

## Synthetic survey and target grid

In [6]:
json_file = results_dir / "synthetic-surveys.json"
tex_file = manuscript_dir / "synthetic-surveys.tex"

with open(json_file, "r") as f:
    variables = json.loads(f.read())

In [7]:
units = variables.copy()

units["n_prisms"] = None
units["model_easting"] = "m"
units["model_northing"] = "m"
units["model_depth"] = "m"
units["model_min_density"] = "kg m-3"
units["model_max_density"] = "kg m-3"
units["survey_easting"] = "m"
units["survey_northing"] = "m"
units["survey_noise"] = "mGal"
units["ground_survey_points"] = None
units["ground_survey_min_height"] = "m"
units["ground_survey_max_height"] = "m"
units["airborne_survey_points"] = None
units["airborne_survey_min_height"] = "m"
units["airborne_survey_max_height"] = "m"
units["target_height"] = "m"
units["target_spacing"] = "m"
units["target_easting_size"] = None
units["target_northing_size"] = None

In [8]:
latex_variables = [
    create_latex_variable(key, value, unit=units[key])
    for key, value in variables.items()
]
display(latex_variables)

['\\newcommand{\\NPrisms}{64}',
 '\\newcommand{\\ModelEasting}{$111319 \\, \\text{m}$}',
 '\\newcommand{\\ModelNorthing}{$111319 \\, \\text{m}$}',
 '\\newcommand{\\ModelDepth}{$10000 \\, \\text{m}$}',
 '\\newcommand{\\ModelMinDensity}{$-900 \\, \\text{kg} \\, \\text{m}^{-3}$}',
 '\\newcommand{\\ModelMaxDensity}{$500 \\, \\text{kg} \\, \\text{m}^{-3}$}',
 '\\newcommand{\\SurveyEasting}{$111319 \\, \\text{m}$}',
 '\\newcommand{\\SurveyNorthing}{$110576 \\, \\text{m}$}',
 '\\newcommand{\\SurveyNoise}{$1 \\, \\text{mGal}$}',
 '\\newcommand{\\GroundSurveyPoints}{963}',
 '\\newcommand{\\GroundSurveyMinHeight}{$0 \\, \\text{m}$}',
 '\\newcommand{\\GroundSurveyMaxHeight}{$2052.2 \\, \\text{m}$}',
 '\\newcommand{\\AirborneSurveyPoints}{5673}',
 '\\newcommand{\\AirborneSurveyMinHeight}{$359 \\, \\text{m}$}',
 '\\newcommand{\\AirborneSurveyMaxHeight}{$1255 \\, \\text{m}$}',
 '\\newcommand{\\TargetHeight}{$2000 \\, \\text{m}$}',
 '\\newcommand{\\TargetSpacing}{$2 \\, \\text{m}$}',
 '\\newcommand{\

In [9]:
with open(tex_file, "w") as f:
    f.write("\n".join(latex_variables))

## Gridding ground survey

In [10]:
json_file = results_dir / "parameters-ground-survey.json"
tex_file = manuscript_dir / "parameters-ground-survey.tex"

with open(json_file, "r") as f:
    parameters = json.loads(f.read())

In [11]:
latex_variables = parameters_to_latex(parameters, survey="ground")
display(latex_variables)

['\\newcommand{\\GroundSourceBelowDataConstantDepthDamping}{10$^{-4}$, 10$^{-3}$,$\\dots$, 10$^{2}$}',
 '\\newcommand{\\GroundSourceBelowDataConstantDepthDepth}{1000 to 17000, step size 2000}',
 '\\newcommand{\\GroundSourceBelowDataRelativeDepthDamping}{10$^{-4}$, 10$^{-3}$,$\\dots$, 10$^{2}$}',
 '\\newcommand{\\GroundSourceBelowDataRelativeDepthDepth}{1000 to 17000, step size 2000}',
 '\\newcommand{\\GroundSourceBelowDataVariableDepthDamping}{10$^{-4}$, 10$^{-3}$,$\\dots$, 10$^{2}$}',
 '\\newcommand{\\GroundSourceBelowDataVariableDepthDepthFactor}{0.1, 0.5, 1, 2, 3, 4, 5 and 6}',
 '\\newcommand{\\GroundSourceBelowDataVariableDepthDepth}{0 to 1400, step size 200}',
 '\\newcommand{\\GroundSourceBelowDataVariableDepthKNearest}{1, 5, 10 and 15}',
 '\\newcommand{\\GroundBlockAveragedSourcesConstantDepthDamping}{10$^{-4}$, 10$^{-3}$,$\\dots$, 10$^{2}$}',
 '\\newcommand{\\GroundBlockAveragedSourcesConstantDepthDepth}{1000 to 17000, step size 2000}',
 '\\newcommand{\\GroundBlockAveragedSource

In [12]:
with open(tex_file, "w") as f:
    f.write("\n".join(latex_variables))

### Ground survey: best predictions

In [13]:
best_predictions = [
    xr.open_dataset(f)
    for f in ground_results_dir.iterdir()
    if "best_predictions" in f.name
]

latex_variables = []
for dataset in best_predictions:
    for array in dataset:
        latex_variables.extend(
            best_parameters_to_latex(dataset[array].attrs, survey="ground")
        )

tex_file = manuscript_dir / "best-parameters-ground-survey.tex"
with open(tex_file, "w") as f:
    f.write("\n".join(latex_variables))

## Gridding airborne survey

In [14]:
json_file = results_dir / "parameters-airborne-survey.json"
tex_file = manuscript_dir / "parameters-airborne-survey.tex"

with open(json_file, "r") as f:
    parameters = json.loads(f.read())

In [15]:
latex_variables = parameters_to_latex(parameters, survey="airborne")
display(latex_variables)

['\\newcommand{\\AirborneSourceBelowDataConstantDepthDamping}{10$^{-4}$, 10$^{-3}$,$\\dots$, 10$^{2}$}',
 '\\newcommand{\\AirborneSourceBelowDataConstantDepthDepth}{1000 to 17000, step size 2000}',
 '\\newcommand{\\AirborneSourceBelowDataRelativeDepthDamping}{10$^{-4}$, 10$^{-3}$,$\\dots$, 10$^{2}$}',
 '\\newcommand{\\AirborneSourceBelowDataRelativeDepthDepth}{1000 to 17000, step size 2000}',
 '\\newcommand{\\AirborneSourceBelowDataVariableDepthDamping}{10$^{-4}$, 10$^{-3}$,$\\dots$, 10$^{2}$}',
 '\\newcommand{\\AirborneSourceBelowDataVariableDepthDepthFactor}{1 to 6, step size 1}',
 '\\newcommand{\\AirborneSourceBelowDataVariableDepthDepth}{50 to 1450, step size 200}',
 '\\newcommand{\\AirborneSourceBelowDataVariableDepthKNearest}{1, 5, 10 and 15}',
 '\\newcommand{\\AirborneBlockAveragedSourcesConstantDepthDamping}{10$^{-4}$, 10$^{-3}$,$\\dots$, 10$^{2}$}',
 '\\newcommand{\\AirborneBlockAveragedSourcesConstantDepthDepth}{1000 to 17000, step size 2000}',
 '\\newcommand{\\AirborneBlockA

In [16]:
with open(tex_file, "w") as f:
    f.write("\n".join(latex_variables))

### Airborne survey: best predictions

In [17]:
best_predictions = [
    xr.open_dataset(f)
    for f in airborne_results_dir.iterdir()
    if "best_predictions" in f.name
]

latex_variables = []
for dataset in best_predictions:
    for array in dataset:
        latex_variables.extend(
            best_parameters_to_latex(dataset[array].attrs, survey="airborne")
        )

tex_file = manuscript_dir / "best-parameters-airborne-survey.tex"
with open(tex_file, "w") as f:
    f.write("\n".join(latex_variables))

## Gradient boosted sources

In [18]:
json_file = results_dir / "boost-overlapping.json"
tex_file = manuscript_dir / "boost-overlapping.tex"

with open(json_file, "r") as f:
    variables = json.loads(f.read())

units["boost_overlapping_window_size"] = "m"

In [19]:
latex_variables = [
    create_latex_variable(key, value, unit=units[key])
    for key, value in variables.items()
]
display(latex_variables)

['\\newcommand{\\BoostOverlappingWindowSize}{$30000 \\, \\text{m}$}']

In [20]:
with open(tex_file, "w") as f:
    f.write("\n".join(latex_variables))

## Australia gravity

In [21]:
json_file = results_dir / "australia.json"
tex_file = manuscript_dir / "australia.tex"

with open(json_file, "r") as f:
    variables = json.loads(f.read())

variables["australia_eql_window_size"] *= 1e-3

units["australia_eql_depth"] = "m"
units["australia_eql_damping"] = None
units["australia_eql_spacing"] = "m"
units["australia_eql_window_size"] = "km"
units["australia_eql_n_sources"] = None
units["australia_eql_grid_n_longitude"] = None
units["australia_eql_grid_n_latitude"] = None
units["australia_eql_grid_height"] = "m"

In [22]:
latex_variables = [
    create_latex_variable(key, value, unit=units[key])
    for key, value in variables.items()
]
display(latex_variables)

['\\newcommand{\\AustraliaEqlDepth}{$4000 \\, \\text{m}$}',
 '\\newcommand{\\AustraliaEqlDamping}{100}',
 '\\newcommand{\\AustraliaEqlSpacing}{$1800 \\, \\text{m}$}',
 '\\newcommand{\\AustraliaEqlWindowSize}{$225 \\, \\text{km}$}',
 '\\newcommand{\\AustraliaEqlNSources}{796744}',
 '\\newcommand{\\AustraliaEqlGridNLongitude}{2442}',
 '\\newcommand{\\AustraliaEqlGridNLatitude}{2085}',
 '\\newcommand{\\AustraliaEqlGridHeight}{$2127.58 \\, \\text{m}$}']

In [23]:
with open(tex_file, "w") as f:
    f.write("\n".join(latex_variables))